# Proyecto de aprendizaje automático de extremo a extremo

### Trabajar con datos reales

Si queremos obtener resultados confiables, es fundamental trabajar con datos reales y no con datos inventados. Los modelos de aprendizaje automático solo pueden aprender patrones útiles si la información que reciben representa el mundo tal como es, con sus imperfecciones, ruido y complejidad.

Afortunadamente, existen múltiples plataformas que recopilan y publican conjuntos de datos abiertos. Muchas de ellas cuentan con comunidades activas que actualizan y comparten información de forma continua, lo que permite acceder a datos variados y de buena calidad.

Algunos de los repositorios de datos abiertos más populares son:

* **OpenML.org**
* **Kaggle.com**
* **PapersWithCode.com**

En este capítulo trabajaremos con el conjunto de datos **California Housing Prices**, que contiene información de un censo realizado en 1990. Nuestro objetivo será utilizar estos datos para construir un modelo capaz de predecir el precio de las viviendas en distintas zonas de California.

### Pipelines

Una **pipeline** es una secuencia de etapas automatizadas que transforman datos de forma organizada y reproducible.

En sistemas de *machine learning*, las pipelines son muy comunes cuando se necesita transformar, limpiar o manipular grandes volúmenes de datos antes de entrenar un modelo. En lugar de hacer cada paso manualmente, se encadenan procesos que se ejecutan uno tras otro.

Cada componente de la pipeline suele cumplir una función específica:

* Extraer datos (por ejemplo, desde una base de datos o un archivo).
* Procesarlos (limpieza, normalización, transformación, generación de características).
* Guardar el resultado en otro almacenamiento para que la siguiente etapa lo utilice.

En muchos sistemas modernos, estas etapas pueden ejecutarse de forma asíncrona o distribuida, lo que permite manejar grandes cantidades de información de manera eficiente.


### Antes de comenzar

Antes de iniciar un proyecto real, debemos preguntarnos cómo vamos a abordar el problema. No se trata solo de “entrenar algo”, sino de elegir correctamente el enfoque desde el principio.

En el caso de **California Housing Prices**, podemos plantearnos varias preguntas clave:

* ¿Qué tipo de aprendizaje necesitamos?
  ¿Supervisado, no supervisado, semi-supervisado, self-supervised o reinforcement learning?

* ¿Usaremos *batch learning* o *online learning*?

Empecemos por lo más sencillo:
¿Nuestro sistema recibirá un flujo constante de datos nuevos para actualizar el modelo? No. El conjunto de datos es histórico y está cerrado; queremos entrenar el modelo para hacer predicciones sobre ese mismo tipo de información. No necesitamos que el modelo se actualice continuamente con datos nuevos, así que utilizaremos **batch learning**.

Ahora, ¿qué tipo de aprendizaje es?
Tenemos el precio de las casas (la variable objetivo) y múltiples características asociadas a cada vivienda. Es decir, conocemos la respuesta correcta durante el entrenamiento. Por lo tanto, se trata claramente de **aprendizaje supervisado**.

Finalmente, ¿qué tipo de modelo utilizaremos?
El comportamiento que buscamos modelar es esencialmente lineal. Sin embargo, no tenemos una sola variable explicativa, sino varias características que influyen en el precio. Por esta razón, no basta con una regresión lineal simple: utilizaremos **regresión lineal múltiple**, que nos permite considerar múltiples variables de entrada al mismo tiempo.

### Métrica de desempeño

Para asegurarnos de que nuestro modelo realmente funciona bien, necesitamos definir una **métrica de desempeño**. Sin una métrica, no hay forma rigurosa de saber si el modelo está aprendiendo o simplemente ajustándose al azar.

En problemas de **regresión lineal**, una de las métricas más utilizadas es el **Root Mean Square Error (RMSE)**. Esta métrica nos indica, en promedio, qué tan lejos están nuestras predicciones de los valores reales.

Una característica importante del RMSE es que **penaliza más fuertemente los errores grandes**, ya que eleva cada error al cuadrado antes de promediarlos. Esto lo hace especialmente útil cuando queremos evitar predicciones con fallos extremos.

La fórmula del RMSE es:

$$
RMSE = \sqrt{\frac{1}{m} \sum_{i=1}^{m} \left( \hat{y}^{(i)} - y^{(i)} \right)^2}
$$

donde:

* ( $m$ ) es el número total de ejemplos,
* ( $\hat{y}^{(i)} = h_\theta(x^{(i)}) $) es la predicción generada por la hipótesis para el ejemplo ( i ),
* ( $y^{(i)}$ ) es el valor real correspondiente.

Aquí es importante recordar que ( $h_\theta(\cdot)$ ) es la **función hipótesis**, mientras que ( $\hat{y}^{(i)}$ ) es el resultado de evaluarla en un ejemplo específico.

En la mayoría de problemas reales, las entradas no son números individuales sino **vectores de características**. En ese caso, la hipótesis en regresión lineal se define como:

$$
h_\theta(x) = \theta^T x
$$

Por lo tanto, la predicción para el ejemplo ( i ) es:

$$
\hat{y}^{(i)} = \theta^T x^{(i)}
$$

Y el RMSE puede escribirse como:

$$
RMSE = \sqrt{\frac{1}{m} \sum_{i=1}^{m} \left( \theta^T x^{(i)} - y^{(i)} \right)^2}
$$

Sin embargo, en algunos contextos puede ser preferible utilizar otra función de error. Por ejemplo, podemos considerar el **Mean Absolute Error (MAE)**, también llamado **promedio de desviación absoluta**.

Su expresión es:

$$
MAE = \frac{1}{m} \sum_{i=1}^{m} \left| \hat{y}^{(i)} - y^{(i)} \right|
$$

A diferencia del RMSE, aquí **no elevamos al cuadrado los errores**, sino que tomamos su valor absoluto. Esto hace que el MAE sea menos sensible a valores atípicos (outliers), ya que no amplifica los errores grandes.

### Interpretación geométrica

Tanto el **RMSE** como el **MAE** pueden interpretarse como formas de medir la distancia entre dos vectores:

* El vector de predicciones
* El vector de valores reales (etiquetas)

Si definimos:

$$
\mathbf{\hat{y}} =
\begin{bmatrix}
\hat{y}^{(1)} \
\hat{y}^{(2)} \
\vdots \
\hat{y}^{(m)}
\end{bmatrix}
\quad
\text{y}
\quad
\mathbf{y} =
\begin{bmatrix}
y^{(1)} \
y^{(2)} \
\vdots \
y^{(m)}
\end{bmatrix}
$$

Entonces estamos midiendo la distancia entre esos dos vectores en ( $\mathbb{R}^m$ ).

* Cuando elevamos al cuadrado y sumamos, estamos usando la **norma euclidiana** ( $\ell_2$ ).
* Cuando usamos valores absolutos, estamos usando la **norma Manhattan** ( $\ell_1$ ).

La norma ( $\ell_2$ ) mide la distancia “en línea recta” entre dos puntos (como un dron que vuela directo).

La norma ( $\ell_1$ ) mide la distancia como si te movieras por una ciudad en cuadrícula, avanzando solo en direcciones horizontales y verticales (de ahí el nombre Manhattan).

Estas dos son casos particulares de una familia más general llamada **norma ( $\ell_k$ )**:

$$
|v|_k =
\left(
|v_1|^k + |v_2|^k + \dots + |v_n|^k
\right)^{1/k}
$$

Cuanto mayor sea el valor de ( $k$ ), **más peso tendrán los valores grandes del vector**, y menor influencia relativa tendrán los pequeños.

En otras palabras:
si aumentas ( $k$ ), el modelo se vuelve cada vez más intolerante a errores grandes.


# Empecemos con el código

## Descargar los datos

En un entorno real, lo más común es que la información esté almacenada en una **base de datos relacional**, como PostgreSQL o MySQL, o incluso en sistemas distribuidos más complejos.

Sin embargo, en este proyecto simplificaremos el proceso: trabajaremos con un archivo **CSV** que ya estará disponible dentro del repositorio. Esto nos permitirá concentrarnos en el análisis y el modelado sin distraernos con la infraestructura.

Aunque en un escenario profesional podríamos descargar los datos dinámicamente desde una API o una base de datos externa, para efectos del curso mantendremos todo organizado localmente.

Crearemos una función encargada de:

1. Localizar el archivo dentro del proyecto.
2. Leer el archivo CSV.
3. Cargar los datos en memoria para comenzar a explorarlos.